In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !pip install tensorflow tensorflow_hub librosa scikit-learn matplotlib seaborn joblib

In [8]:
import os
import numpy as np
import librosa
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

ModuleNotFoundError: No module named 'librosa'

In [4]:
# Step 1: Load YAMNet model from TensorFlow Hub
print("Loading YAMNet model...")
yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')
print("YAMNet loaded.")

Loading YAMNet model...


NameError: name 'hub' is not defined

In [ ]:
# Step 2: Define function to extract YAMNet embeddings from an audio file
def extract_yamnet_embedding(file_path, target_sr=16000):
    """
    Loads an audio file, resamples it to target_sr, and extracts the mean YAMNet embedding.
    """
    waveform, sr = librosa.load(file_path, sr=target_sr, mono=True)
    waveform = waveform.astype(np.float32)
    scores, embeddings, spectrogram = yamnet_model(waveform)
    mean_embedding = np.mean(embeddings.numpy(), axis=0)
    return mean_embedding

In [ ]:
data_dir = '/home/ankit/WindowsFuneral/Hackathons/KU-HACKFEST-2025/acoustic-system/LoRa-Net/Forest_Datasets'
audio_extensions = ('.wav', '.mp3')

file_paths = []
labels = []

# Directly iterate through sound class folders
for subfolder in os.listdir(data_dir):
    subfolder_path = os.path.join(data_dir, subfolder)
    if os.path.isdir(subfolder_path):
        for file in os.listdir(subfolder_path):
            if file.lower().endswith(audio_extensions):
                file_paths.append(os.path.join(subfolder_path, file))
                labels.append(subfolder)  # Use folder name as label

print(f"Total audio files found: {len(file_paths)}")
print(f"Unique sub-labels: {len(set(labels))} -> {set(labels)}")

In [ ]:
# Step 4: Extract embeddings for all audio files (this may take time)
print("Extracting YAMNet embeddings for all audio files...")
embeddings = []
failed_files = []
for fp in file_paths:
    try:
        emb = extract_yamnet_embedding(fp)
        embeddings.append(emb)
    except Exception as e:
        print(f"Failed to process {fp}: {e}")
        failed_files.append(fp)

embeddings = np.array(embeddings)
print(f"Extracted embeddings shape: {embeddings.shape}")

# Remove failed files from labels
if failed_files:
    print(f"Removing {len(failed_files)} failed files from labels.")
    labels = [label for fp, label in zip(file_paths, labels) if fp not in failed_files]

labels = np.array(labels)

In [ ]:
# Step 5: Encode labels to one-hot vectors
label_binarizer = LabelBinarizer()
one_hot_labels = label_binarizer.fit_transform(labels)
class_names = label_binarizer.classes_
print(f"Classes: {class_names}")
print(f"One-hot labels shape: {one_hot_labels.shape}")

In [ ]:
# Step 6: Split dataset into train and test sets (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    embeddings, one_hot_labels, test_size=0.2, random_state=42, stratify=one_hot_labels)

print(f"Training samples: {X_train.shape[0]}, Testing samples: {X_test.shape[0]}")

In [ ]:
# Step 7: Build Keras classifier model with softmax output
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

num_classes = len(class_names)

model = Sequential([
    Dense(128, activation='relu', input_shape=(1024,)),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
from sklearn.utils import class_weight
import numpy as np

# Assuming y_train are integer labels of shape (n_samples,)
# Need to use the original labels before one-hot encoding for compute_class_weight
# Assuming 'labels' variable contains the original string labels corresponding to the training data
# We need to select the labels corresponding to the training split.
# Since train_test_split was stratified on the one-hot labels, we can use the indices
# to get the corresponding original labels. However, a simpler way is to use the original
# 'labels' array and split it with the same random_state and stratify settings.

# Re-split the original labels to get the training labels
from sklearn.model_selection import train_test_split

# Assuming the original 'labels' variable is still available from Step 3
# If not, you would need to reconstruct or save it earlier.
# Let's assume 'labels' from cell zXBhN5OGr2D2 is available.

# Since the split in cell HzGQDz__sMGa was stratified on one_hot_labels,
# we can perform the same split on the original 'labels' array to get y_train_original.
# This ensures the correct correspondence between X_train and the original labels.
X_train_dummy, X_test_dummy, y_train_original, y_test_original = train_test_split(
    embeddings, labels, test_size=0.2, random_state=42, stratify=one_hot_labels)


classes = np.unique(y_train_original)
weights = class_weight.compute_class_weight('balanced', classes=classes, y=y_train_original)
class_weight_dict = dict(zip(classes, weights))

print("Class weights calculated:")
for cls, weight in class_weight_dict.items():
    print(f"  {cls}: {weight:.4f}")

In [ ]:
# Step 8: Train the model
print("Training the classifier...")
history = model.fit(X_train, y_train,
                    epochs=30,
                    batch_size=32,
                    class_weight=class_weight_dict,
                    validation_split=0.2,
                    verbose=1)

In [ ]:
# Step 9: Evaluate the model on the test set
print("\nEvaluating on test set...")
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

y_pred_probs = model.predict(X_test)
y_pred_classes = y_pred_probs.argmax(axis=1)
y_true_classes = y_test.argmax(axis=1)

print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=class_names))

In [ ]:
# Plot confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Step 10: Save the trained model and label binarizer
model_save_path = '/content/multi_class_audio_classifier.h5'
model.save(model_save_path)
print(f"Model saved to: {model_save_path}")

label_binarizer_path = '/content/label_binarizer.pkl'
joblib.dump(label_binarizer, label_binarizer_path)
print(f"Label binarizer saved to: {label_binarizer_path}")

In [ ]:
# Step 11: Function to predict sub-label on new audio file
# def predict_audio_file(file_path, yamnet_model, keras_model, label_binarizer_obj, target_sr=16000):
#     """
#     Predict the sub-label of a given audio file.
#     """
#     waveform, sr = librosa.load(file_path, sr=target_sr, mono=True)
#     waveform = waveform.astype(np.float32)
#     scores, embeddings, spectrogram = yamnet_model(waveform)
#     mean_embedding = np.mean(embeddings.numpy(), axis=0).reshape(1, -1)

#     probs = keras_model.predict(mean_embedding)[0]
#     pred_index = np.argmax(probs)
#     pred_label = label_binarizer_obj.classes_[pred_index]
#     confidence = probs[pred_index]

#     print(f"Predicted sub-label: {pred_label} with confidence {confidence:.3f}")
#     return pred_label, confidence

## batch_predict_folder

In [ ]:
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt

def aggregate_probabilities_over_clip(file_path, yamnet_model, keras_model, label_binarizer,
                                      window_size=2.0, hop_size=1.0, sr=16000):
    """
    For a given audio file, compute softmax probabilities per sliding window,
    aggregate over entire clip, and return average probabilities per class.
    """
    waveform, _ = librosa.load(file_path, sr=sr, mono=True)
    waveform = waveform.astype(np.float32)

    window_samples = int(window_size * sr)
    hop_samples = int(hop_size * sr)

    num_windows = max(1, (len(waveform) - window_samples) // hop_samples + 1)

    prob_sums = np.zeros(len(label_binarizer.classes_))

    for i in range(num_windows):
        start_sample = i * hop_samples
        end_sample = start_sample + window_samples
        window_waveform = waveform[start_sample:end_sample]

        if len(window_waveform) < window_samples:
            window_waveform = np.pad(window_waveform, (0, window_samples - len(window_waveform)))

        scores, embeddings, _ = yamnet_model(window_waveform)
        mean_embedding = np.mean(embeddings.numpy(), axis=0).reshape(1, -1)

        probs = keras_model.predict(mean_embedding)[0]
        prob_sums += probs

    avg_probs = prob_sums / num_windows
    percentages = avg_probs * 100
    return percentages

def plot_sound_distribution(percentages, class_names, file_name):
    """
    Plot a bar chart of sound class percentages for a single audio file.
    """
    sorted_indices = np.argsort(percentages)[::-1]
    sorted_labels = class_names[sorted_indices]
    sorted_percentages = percentages[sorted_indices]

    plt.figure(figsize=(12, 6))
    plt.bar(sorted_labels, sorted_percentages, color='skyblue')
    plt.xticks(rotation=45, ha='right')
    plt.ylabel('Percentage (%)')
    plt.title(f'Sound Class Distribution for {file_name}')
    plt.tight_layout()
    plt.show()

def batch_process_folder(test_folder, yamnet_model, keras_model, label_binarizer):
    """
    Process all audio files in a folder, aggregate probabilities, and plot distributions.
    """
    audio_extensions = ('.wav', '.mp3', '.flac', '.ogg', '.m4a')
    for root, _, files in os.walk(test_folder):
        for file in files:
            if file.lower().endswith(audio_extensions):
                file_path = os.path.join(root, file)
                print(f"\nProcessing file: {file}")
                percentages = aggregate_probabilities_over_clip(file_path, yamnet_model, keras_model, label_binarizer)
                for label, pct in zip(label_binarizer.classes_, percentages):
                    print(f"{label}: {pct:.2f}%")
                plot_sound_distribution(percentages, label_binarizer.classes_, file)

In [ ]:
# Usage example:
test_folder_path = '/content/drive/MyDrive/testme'  # Your test folder path
batch_process_folder(test_folder_path, yamnet_model, model, label_binarizer)

## batch_process_folder_with_unknown

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# def aggregate_probabilities_with_unknown(file_path, yamnet_model, keras_model, label_binarizer,
#                                          window_size=2.0, hop_size=1.0, sr=16000, unknown_threshold=0.4):
#     waveform, _ = librosa.load(file_path, sr=sr, mono=True)
#     waveform = waveform.astype(np.float32)

#     window_samples = int(window_size * sr)
#     hop_samples = int(hop_size * sr)

#     num_windows = max(1, (len(waveform) - window_samples) // hop_samples + 1)

#     prob_sums = np.zeros(len(label_binarizer.classes_))

#     for i in range(num_windows):
#         start_sample = i * hop_samples
#         end_sample = start_sample + window_samples
#         window_waveform = waveform[start_sample:end_sample]

#         if len(window_waveform) < window_samples:
#             window_waveform = np.pad(window_waveform, (0, window_samples - len(window_waveform)))

#         scores, embeddings, _ = yamnet_model(window_waveform)
#         mean_embedding = np.mean(embeddings.numpy(), axis=0).reshape(1, -1)

#         probs = keras_model.predict(mean_embedding)[0]
#         prob_sums += probs

#     avg_probs = prob_sums / num_windows

#     max_prob = np.max(avg_probs)
#     unknown_prob = 0.0
#     if max_prob < unknown_threshold:
#         unknown_prob = 1.0 - np.sum(avg_probs)
#         # Clip unknown_prob to be non-negative (in case sum(avg_probs) > 1 due to numerical issues)
#         unknown_prob = max(0.0, unknown_prob)

#     # Prepare final classes and percentages including unknown
#     final_classes = list(label_binarizer.classes_) + ['unknown']
#     final_percentages = list(avg_probs * 100) + [unknown_prob * 100]

#     return final_classes, final_percentages

# def plot_distribution_with_unknown(classes, percentages, file_name):
#     sorted_indices = np.argsort(percentages)[::-1]
#     sorted_labels = np.array(classes)[sorted_indices]
#     sorted_percentages = np.array(percentages)[sorted_indices]

#     plt.figure(figsize=(12, 6))
#     plt.bar(sorted_labels, sorted_percentages, color='coral')
#     plt.xticks(rotation=45, ha='right')
#     plt.ylabel('Percentage (%)')
#     plt.title(f'Sound Class Distribution with Unknown for {file_name}')
#     plt.tight_layout()
#     plt.show()

# def batch_process_folder_with_unknown(test_folder, yamnet_model, keras_model, label_binarizer,
#                                       unknown_threshold=0.4):
#     audio_extensions = ('.wav', '.mp3', '.flac', '.ogg', '.m4a')
#     for root, _, files in os.walk(test_folder):
#         for file in files:
#             if file.lower().endswith(audio_extensions):
#                 file_path = os.path.join(root, file)
#                 print(f"\nProcessing file: {file}")
#                 classes, percentages = aggregate_probabilities_with_unknown(
#                     file_path, yamnet_model, keras_model, label_binarizer,
#                     unknown_threshold=unknown_threshold)
#                 for label, pct in zip(classes, percentages):
#                     print(f"{label}: {pct:.2f}%")
#                 plot_distribution_with_unknown(classes, percentages, file)

# # Usage example:
# test_folder_path = '/content/drive/MyDrive/testme'
# batch_process_folder_with_unknown(test_folder_path, yamnet_model, model, label_binarizer, unknown_threshold=0.4)


## batch_process_folder_with_confidence

In [ ]:
# import os
# import librosa
# import numpy as np
# import matplotlib.pyplot as plt

# def aggregate_probabilities_with_confidence_filter(file_path, yamnet_model, keras_model, label_binarizer,
#                                                   window_size=2.0, hop_size=1.0, sr=16000,
#                                                   confidence_threshold=0.6, unknown_threshold=0.4):
#     """
#     Aggregate softmax probabilities over sliding windows in an audio file,
#     only including windows where max class probability >= confidence_threshold.
#     Assign residual probability to 'unknown' class if confidence is low.
#     """
#     waveform, _ = librosa.load(file_path, sr=sr, mono=True)
#     waveform = waveform.astype(np.float32)

#     window_samples = int(window_size * sr)
#     hop_samples = int(hop_size * sr)

#     num_windows = max(1, (len(waveform) - window_samples) // hop_samples + 1)

#     prob_sums = np.zeros(len(label_binarizer.classes_))
#     confident_window_count = 0

#     for i in range(num_windows):
#         start_sample = i * hop_samples
#         end_sample = start_sample + window_samples
#         window_waveform = waveform[start_sample:end_sample]

#         if len(window_waveform) < window_samples:
#             window_waveform = np.pad(window_waveform, (0, window_samples - len(window_waveform)))

#         scores, embeddings, _ = yamnet_model(window_waveform)
#         mean_embedding = np.mean(embeddings.numpy(), axis=0).reshape(1, -1)

#         probs = keras_model.predict(mean_embedding)[0]
#         max_prob = np.max(probs)

#         if max_prob >= confidence_threshold:
#             prob_sums += probs
#             confident_window_count += 1
#         else:
#             # Window considered uncertain; ignored in known class aggregation
#             pass

#     if confident_window_count > 0:
#         avg_probs = prob_sums / confident_window_count
#     else:
#         avg_probs = np.zeros(len(label_binarizer.classes_))

#     unknown_prob = 0.0
#     # If too few confident windows or max avg prob low, assign unknown prob
#     if confident_window_count < num_windows or np.max(avg_probs) < unknown_threshold:
#         unknown_prob = 1.0 - np.sum(avg_probs)
#         unknown_prob = max(0.0, unknown_prob)

#     final_classes = list(label_binarizer.classes_) + ['unknown']
#     final_percentages = list(avg_probs * 100) + [unknown_prob * 100]

#     return final_classes, final_percentages

# def plot_distribution_with_unknown(classes, percentages, file_name):
#     """
#     Plot a bar chart of sound class percentages including unknown class.
#     """
#     sorted_indices = np.argsort(percentages)[::-1]
#     sorted_labels = np.array(classes)[sorted_indices]
#     sorted_percentages = np.array(percentages)[sorted_indices]

#     plt.figure(figsize=(12, 6))
#     plt.bar(sorted_labels, sorted_percentages, color='mediumseagreen')
#     plt.xticks(rotation=45, ha='right')
#     plt.ylabel('Percentage (%)')
#     plt.title(f'Sound Class Distribution with Unknown for {file_name}')
#     plt.tight_layout()
#     plt.show()

# def batch_process_folder_with_confidence(test_folder, yamnet_model, keras_model, label_binarizer,
#                                          confidence_threshold=0.6, unknown_threshold=0.4):
#     """
#     Process all audio files in a folder with confidence filtering and plot distributions.
#     """
#     audio_extensions = ('.wav', '.mp3', '.flac', '.ogg', '.m4a')
#     for root, _, files in os.walk(test_folder):
#         for file in files:
#             if file.lower().endswith(audio_extensions):
#                 file_path = os.path.join(root, file)
#                 print(f"\nProcessing file: {file}")
#                 classes, percentages = aggregate_probabilities_with_confidence_filter(
#                     file_path, yamnet_model, keras_model, label_binarizer,
#                     confidence_threshold=confidence_threshold,
#                     unknown_threshold=unknown_threshold)
#                 for label, pct in zip(classes, percentages):
#                     print(f"{label}: {pct:.2f}%")
#                 plot_distribution_with_unknown(classes, percentages, file)

# # Usage example:
# test_folder_path = '/content/drive/MyDrive/testme'  # Update to your test folder path
# batch_process_folder_with_confidence(test_folder_path, yamnet_model, model, label_binarizer,
#                                      confidence_threshold=0.15, unknown_threshold=0.4)


In [ ]:
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt

def aggregate_probabilities_with_confidence_filter(file_path, yamnet_model, keras_model, label_binarizer,
                                                  window_size=2.0, hop_size=1.0, sr=16000,
                                                  confidence_threshold=0.6):
    """
    Aggregate softmax probabilities over sliding windows in an audio file,
    only including windows where max class probability >= confidence_threshold.
    Does NOT assign residual probability to 'unknown' class.
    """
    waveform, _ = librosa.load(file_path, sr=sr, mono=True)
    waveform = waveform.astype(np.float32)

    window_samples = int(window_size * sr)
    hop_samples = int(hop_size * sr)

    num_windows = max(1, (len(waveform) - window_samples) // hop_samples + 1)

    prob_sums = np.zeros(len(label_binarizer.classes_))
    confident_window_count = 0

    for i in range(num_windows):
        start_sample = i * hop_samples
        end_sample = start_sample + window_samples
        window_waveform = waveform[start_sample:end_sample]

        if len(window_waveform) < window_samples:
            window_waveform = np.pad(window_waveform, (0, window_samples - len(window_waveform)))

        scores, embeddings, _ = yamnet_model(window_waveform)
        mean_embedding = np.mean(embeddings.numpy(), axis=0).reshape(1, -1)

        probs = keras_model.predict(mean_embedding)[0]
        max_prob = np.max(probs)

        if max_prob >= confidence_threshold:
            prob_sums += probs
            confident_window_count += 1
        else:
            # Window considered uncertain; ignored in known class aggregation
            pass

    if confident_window_count > 0:
        avg_probs = prob_sums / confident_window_count
    else:
        avg_probs = np.zeros(len(label_binarizer.classes_))

    final_classes = list(label_binarizer.classes_)  # No 'unknown' class added
    final_percentages = list(avg_probs * 100)       # No unknown probability added

    return final_classes, final_percentages


def plot_distribution_with_unknown(classes, percentages, file_name):
    """
    Plot a bar chart of sound class percentages.
    """
    sorted_indices = np.argsort(percentages)[::-1]
    sorted_labels = np.array(classes)[sorted_indices]
    sorted_percentages = np.array(percentages)[sorted_indices]

    plt.figure(figsize=(12, 6))
    plt.bar(sorted_labels, sorted_percentages, color='mediumseagreen')
    plt.xticks(rotation=45, ha='right')
    plt.ylabel('Percentage (%)')
    plt.title(f'Sound Class Distribution for {file_name}')
    plt.tight_layout()
    plt.show()


def batch_process_folder_with_confidence(test_folder, yamnet_model, keras_model, label_binarizer,
                                         confidence_threshold=0.6):
    """
    Process all audio files in a folder with confidence filtering and plot distributions.
    """
    audio_extensions = ('.wav', '.mp3', '.flac', '.ogg', '.m4a')
    for root, _, files in os.walk(test_folder):
        for file in files:
            if file.lower().endswith(audio_extensions):
                file_path = os.path.join(root, file)
                print(f"\nProcessing file: {file}")
                classes, percentages = aggregate_probabilities_with_confidence_filter(
                    file_path, yamnet_model, keras_model, label_binarizer,
                    confidence_threshold=confidence_threshold)
                for label, pct in zip(classes, percentages):
                    print(f"{label}: {pct:.2f}%")
                plot_distribution_with_unknown(classes, percentages, file)


# Usage example:
test_folder_path = '/content/drive/MyDrive/testme'  # Update to your test folder path
batch_process_folder_with_confidence(test_folder_path, yamnet_model, model, label_binarizer,
                                     confidence_threshold=0.15)

## Check the plot 5 second

In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def predict_per_5s_clip(file_path, yamnet_model, keras_model, label_binarizer, clip_duration=5.0, sr=16000):
    """
    Splits the audio file into consecutive non-overlapping 5-second clips,
    predicts softmax probabilities for each clip,
    and returns a list of probability arrays (one per clip).
    """
    waveform, _ = librosa.load(file_path, sr=sr, mono=True)
    waveform = waveform.astype(np.float32)

    clip_samples = int(clip_duration * sr)
    total_samples = len(waveform)
    num_clips = total_samples // clip_samples

    clip_probabilities = []

    for i in range(num_clips):
        start = i * clip_samples
        end = start + clip_samples
        clip_waveform = waveform[start:end]

        # Extract YAMNet embeddings
        scores, embeddings, _ = yamnet_model(clip_waveform)
        mean_embedding = np.mean(embeddings.numpy(), axis=0).reshape(1, -1)

        # Predict softmax probabilities
        probs = keras_model.predict(mean_embedding)[0]
        clip_probabilities.append(probs)

    return clip_probabilities

def plot_grouped_bar_seaborn(clip_probabilities, class_names, clip_duration=5.0):
    """
    Creates a grouped bar chart using Seaborn.
    Each group is a 5-second clip (C1, C2, ...),
    each bar is a class probability.
    """
    # Prepare data for seaborn in long format
    data = []
    for clip_idx, probs in enumerate(clip_probabilities):
        clip_label = f'C{clip_idx+1} ({clip_idx*clip_duration:.0f}-{(clip_idx+1)*clip_duration:.0f}s)'
        for class_idx, class_name in enumerate(class_names):
            data.append({
                'Clip': clip_label,
                'Sound Class': class_name,
                'Probability': probs[class_idx]
            })

    df = pd.DataFrame(data)

    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(14, 7))

    # Create grouped barplot
    ax = sns.barplot(x='Clip', y='Probability', hue='Sound Class', data=df, ci=None)

    ax.set_title('Sound Class Probabilities per 5-Second Clip')
    ax.set_ylabel('Probability')
    ax.set_xlabel('5-Second Clips')
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Sound Class', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

def batch_process_and_plot_seaborn(test_folder, yamnet_model, keras_model, label_binarizer, clip_duration=5.0, sr=16000):
    """
    Process all audio files in a folder, predict per 5-second clip,
    and plot grouped bar charts per file using Seaborn.
    """
    audio_extensions = ('.wav', '.mp3', '.flac', '.ogg', '.m4a')

    for root, _, files in os.walk(test_folder):
        for file in files:
            if file.lower().endswith(audio_extensions):
                file_path = os.path.join(root, file)
                print(f"\nProcessing file: {file}")
                clip_probs = predict_per_5s_clip(file_path, yamnet_model, keras_model, label_binarizer, clip_duration, sr)
                if clip_probs:
                    plot_grouped_bar_seaborn(clip_probs, label_binarizer.classes_, clip_duration)
                else:
                    print("Audio too short for segmentation.")

# Usage example:
test_folder = '/content/drive/MyDrive/testme'
batch_process_and_plot_seaborn(test_folder, yamnet_model, model, label_binarizer, clip_duration=5.0)


## Single inference

In [ ]:
def single_inference_with_confidence(
    file_path, yamnet_model, keras_model, label_binarizer,
    confidence_threshold=0.6
):
    """
    Process a single audio file with confidence filtering and plot the distribution.
    """
    classes, percentages = aggregate_probabilities_with_confidence_filter(
        file_path, yamnet_model, keras_model, label_binarizer,
        confidence_threshold=confidence_threshold
    )
    print(f"\nProcessing file: {os.path.basename(file_path)}")
    for label, pct in zip(classes, percentages):
        print(f"{label}: {pct:.2f}%")
    plot_distribution_with_unknown(classes, percentages, os.path.basename(file_path))

In [ ]:
# Usage example:
audio_file_path = '/content/drive/MyDrive/testme/testing.mpeg'  # Update with your file path
single_inference_with_confidence(
    audio_file_path, yamnet_model, model, label_binarizer,
    confidence_threshold=0.15
)